# Solving models with the COBRA Toolbox in MATLAB.

The [COBRA toolbox](http://dx.doi.org/doi:10.1038/nprot.2011.308) is a commonly used framework for constraint-based modeling in MATLAB. For more information, please visit its [website](https://opencobra.github.io/cobratoolbox/).

Here we will solve all the models in the collection using the COBRA toolbox. The models need to be parsed from SBML/xls into an 'all_models.mat' file, as done by [this script](load_models.ipynb).

The COBRA toolbox also needs to be installed and configured. These paths below may need to be modified:

In [1]:
addpath(genpath('~/cobratoolbox/'));

solvers = {'glpk', 'ibm_cplex', 'gurobi6'};

addpath('/opt/ibm/ILOG/CPLEX_Studio124/cplex/matlab');
addpath('/opt/gurobi602/linux64/matlab');

Load in all the already-parsed models from 'all_models.mat'

In [2]:
loaded_mat = matfile('all_models.mat');
model_ids = who(loaded_mat);
n_models = size(model_ids, 1);
all_models = cell(n_models, 1);
for i=1:n_models
    all_models{i} = loaded_mat.(model_ids{i});
end

Solve all the models

In [3]:
r = zeros(n_models, size(solvers, 2));
for j=1:size(solvers, 2)
    changeCobraSolver(solvers{j});
    for i=1:n_models
        r(i, j) = getfield(optimizeCbModel(all_models{i}), 'f');
    end
end

The solvers all give the same results for each model.

In [4]:
result_table = array2table(r, 'RowNames', model_ids, 'VariableNames', solvers)

result_table = 

                            glpk      ibm_cplex    gurobi6 
                          ________    _________    ________

    AbyMBEL891              119.23      119.23       119.23
    AraGEM                      10          10           10
    GSMN_TB                0.13126     0.13126      0.13126
    PpaMBEL1254             78.701      78.701       78.701
    PpuMBEL1071             132.37      132.37       132.37
    STM_v1_0               0.47783     0.47783      0.47783
    S_coilicolor_fixed      860.09      860.09       860.09
    SpoMBEL1693             63.786      63.786       63.786
    T_Maritima             0.35947     0.35947      0.35947
    VvuMBEL943              96.402      96.402       96.402
    iAC560                  4.0426      4.0426       4.0426
    iAF1260                 0.7367      0.7367       0.7367
    iAF692                 0.02725     0.02725      0.02725
    iAI549                  84.174      84.174       84.174
    iAN840m           

The maximum difference between these solvers is still less than 1e-6

In [5]:
max_diff = max(max(r, [], 2) - min(r, [], 2))

max_diff =

   2.2798e-07